In [1]:
import re
import pandas as pd
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score, recall_score, precision_score
from sklearn.utils import shuffle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
feature_type = '1'

#1 -> just sentence
#2 -> all text
#3 -> all text + other columns

In [3]:
target_type = 3

#1 -> all label
#2 -> pos + neg
#3 -> pos + neg(negative+netural)

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
train = pd.read_csv('train.csv')
test  = pd.read_csv('test.csv')

In [6]:
train.head()

,text,sentence,snp,phenotype,ASSOCIATION,CONFIDENCE
0,OBJECTIVE: Maternal smoking during pregnancy i...,We examined whether a common genetic variant a...,rs1051730,nicotine metabolism,neutral,zero
1,OBJECTIVE: Maternal smoking during pregnancy i...,We examined whether a common genetic variant a...,rs1051730,fetal growth characteristics,positive,weak
2,OBJECTIVE: Maternal smoking during pregnancy i...,RESULTS: Among mothers who did not smoke durin...,rs1051730,fetal growth characteristic,negative,-
3,OBJECTIVE: Maternal smoking during pregnancy i...,Among mothers who continued smoking during pre...,rs1051730,head circumference,negative,-
4,OBJECTIVE: Maternal smoking during pregnancy i...,The T-allele of maternal rs1051730 was associa...,rs1051730,second and third trimester fetal femur length,positive,weak


In [7]:
dic = {"neutral": "negative"}

train.replace({"ASSOCIATION": dic}, inplace=True)
test.replace({"ASSOCIATION": dic}, inplace=True)

In [8]:
train['label'] = train['ASSOCIATION']
test['label']  = test['ASSOCIATION']

In [9]:
train.dropna(subset=['sentence'], inplace=True)
test.dropna(subset=['sentence'], inplace=True)

In [10]:
res = pd.read_excel('res.xlsx')

In [11]:
def vectorize_bow(df_X, cv):
    bow_X = cv.transform(df_X)
    bow_df = pd.DataFrame(data=bow_X.toarray(),
                          columns=[cv.get_feature_names_out()])
    return bow_df

def define_and_run_model(model, X_train, X_test, y_train):
    clf = model
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return y_pred

def show_result(y_pred, y_test):
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred, digits=3))
    print(accuracy_score(y_test, y_pred))
    
def save_res(model_name, feature_type, target_type, y_pred_sr, y_test_sr, idx):
    res_pre = precision_score(y_test_sr, y_pred_sr, average='weighted')
    res_rec = recall_score(y_test_sr, y_pred_sr, average='weighted')
    res_f1 = f1_score(y_test_sr, y_pred_sr, average='weighted')
    res.at[idx, 'model-name'] = model_name
    res.at[idx, 'feature-type'] = feature_type
    res.at[idx, 'target-type'] = target_type
    res.at[idx, 'accuracy'] = accuracy_score(y_test_sr, y_pred_sr)
    res.at[idx, 'precision'] = res_pre
    res.at[idx, 'recall'] = res_rec
    res.at[idx, 'f1-score'] = res_f1

In [12]:
possible_labels = train.label.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

train['label'] = train.label.replace(label_dict)
test['label']  = test.label.replace(label_dict)

In [13]:
df = pd.concat([train, test])

In [14]:
bow_vectorizer = CountVectorizer(lowercase=False, ngram_range=(1, 1), max_df=.80, min_df=1)
bow_vectorizer.fit(df['sentence'])

CountVectorizer(lowercase=False, max_df=0.8)

In [15]:
train_bow_df  = vectorize_bow(list(train['sentence']), bow_vectorizer)
test_bow_df   = vectorize_bow(list(test['sentence']), bow_vectorizer)

In [16]:
X_train = train_bow_df
y_train = train['label']

X_test = test_bow_df
y_test = test['label']

In [17]:
X_train.shape

(934, 2902)

In [18]:
y_train.shape

(934,)

In [19]:
X_test.shape

(365, 2902)

In [20]:
y_test.shape

(365,)

In [21]:
train['ASSOCIATION'].value_counts()

positive    701
negative    233
Name: ASSOCIATION, dtype: int64

In [22]:
test['ASSOCIATION'].value_counts()

negative    195
positive    170
Name: ASSOCIATION, dtype: int64

In [23]:
%%time
smodel = SVC(kernel='rbf')
smodel.fit(X_train, y_train)
pred_values_svm = smodel.predict(X_test)
show_result(pred_values_svm, y_test)
save_res('svm', feature_type, target_type, pred_values_svm, y_test, len(res))

[[ 20 175]
 [  1 169]]
              precision    recall  f1-score   support

           0      0.952     0.103     0.185       195
           1      0.491     0.994     0.658       170

    accuracy                          0.518       365
   macro avg      0.722     0.548     0.421       365
weighted avg      0.738     0.518     0.405       365

0.5178082191780822
Wall time: 1.37 s


In [24]:
%%time
lr = LogisticRegression(random_state=0,
                        penalty='l2',
                        solver='newton-cg',
                        C=1.2,
                        max_iter=20)
lr.fit(X_train, y_train)
pred_values_lr = lr.predict(X_test)
show_result(pred_values_lr, y_test)
save_res('LogisticRegression', feature_type, target_type, pred_values_lr, y_test, len(res))

[[ 63 132]
 [  3 167]]
              precision    recall  f1-score   support

           0      0.955     0.323     0.483       195
           1      0.559     0.982     0.712       170

    accuracy                          0.630       365
   macro avg      0.757     0.653     0.597       365
weighted avg      0.770     0.630     0.590       365

0.6301369863013698
Wall time: 614 ms


In [25]:
%%time
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)
pred_values_rf = model.predict(X_test)
show_result(pred_values_rf, y_test)
save_res('RandomForest', feature_type, target_type, pred_values_rf, y_test, len(res))

[[  9 186]
 [  0 170]]
              precision    recall  f1-score   support

           0      1.000     0.046     0.088       195
           1      0.478     1.000     0.646       170

    accuracy                          0.490       365
   macro avg      0.739     0.523     0.367       365
weighted avg      0.757     0.490     0.348       365

0.4904109589041096
Wall time: 1.48 s


In [26]:
%%time
model = KNeighborsClassifier(n_neighbors=2)
model.fit(X_train, y_train)
pred_values_knn = model.predict(X_test)
show_result(pred_values_knn, y_test)
save_res('knn', feature_type, target_type, pred_values_knn, y_test, len(res))

[[146  49]
 [ 64 106]]
              precision    recall  f1-score   support

           0      0.695     0.749     0.721       195
           1      0.684     0.624     0.652       170

    accuracy                          0.690       365
   macro avg      0.690     0.686     0.687       365
weighted avg      0.690     0.690     0.689       365

0.6904109589041096
Wall time: 178 ms


In [27]:
%%time
model = GaussianNB()
model.fit(X_train, y_train)
pred_values_nb = model.predict(X_test)
show_result(pred_values_nb, y_test)
save_res('GaussianNB', feature_type, target_type, pred_values_nb, y_test, len(res))

[[ 35 160]
 [ 16 154]]
              precision    recall  f1-score   support

           0      0.686     0.179     0.285       195
           1      0.490     0.906     0.636       170

    accuracy                          0.518       365
   macro avg      0.588     0.543     0.460       365
weighted avg      0.595     0.518     0.448       365

0.5178082191780822
Wall time: 168 ms


In [28]:
%%time
model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, y_train)
pred_values_dt = model.predict(X_test)
show_result(pred_values_dt, y_test)
save_res('DecisionTree', feature_type, target_type, pred_values_dt, y_test, len(res))

[[ 68 127]
 [  5 165]]
              precision    recall  f1-score   support

           0      0.932     0.349     0.507       195
           1      0.565     0.971     0.714       170

    accuracy                          0.638       365
   macro avg      0.748     0.660     0.611       365
weighted avg      0.761     0.638     0.604       365

0.6383561643835617
Wall time: 289 ms


In [29]:
%%time
model = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
model.fit(X_train, y_train)
pred_values_gb = model.predict(X_test)
show_result(pred_values_gb, y_test)
save_res('GradientBoosting', feature_type, target_type, pred_values_gb, y_test, len(res))

[[ 48 147]
 [  6 164]]
              precision    recall  f1-score   support

           0      0.889     0.246     0.386       195
           1      0.527     0.965     0.682       170

    accuracy                          0.581       365
   macro avg      0.708     0.605     0.534       365
weighted avg      0.720     0.581     0.524       365

0.5808219178082191
Wall time: 3.76 s


In [30]:
import os
os.remove("res.xlsx") 

res.to_excel('res.xlsx', index=False)